# Import necessary packages

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# Preliminary

In [3]:
#Lets define the URL

url="https://archive.org/details/opensource_movies"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")
#print(soup.prettify()) # print the parsed data of html

# One Movie

## (a) Actual Scrapping of one movie

In [4]:
movies_collection = soup.find("div", attrs={"id": "ikind--downloads"})

# Scrape one movie

aMovie = movies_collection.find_all("div", attrs={"class": "item-ia"})
first = aMovie[17]
#print(first.prettify())
#getting the title of the movie
title = (first.find("div",attrs={"class":"ttl"})).text
title = title.strip()
# Getting the views count of the movie
views = first.find_all("h6",attrs={"class":"stat"})[0].nobr.text
# Stars count
stars = first.find_all("h6",attrs={"class":"stat"})[1].text
# example output of above line if "favorite            12 "
# below line will extract the number from the string above
stars = re.findall("\d+",stars)[0]

comments =  first.find_all("h6",attrs={"class":"stat"})[2].text
comments = re.findall("\d+",comments)[0]

posted_on = ((first.find_all("div",attrs={"class":"hidden-tiles"}))[1]).nobr.text
print(stars)
print(title)
print(views)
print(comments)
print(posted_on)

13
/Damas_BB_28F8B535_D
1.4M
1
May 21, 2013


Now that we know how to scrape one movie we can put all that into a function as follows

In [ ]:
def OneMovie(URL):
    pass

## (b) Writing one movie into .txt file as #-separated 

In [ ]:
# We now write one movie details into text file
# JUst like we have comma for CSV we will use # as the separator
# Lets write the titles first
with open("./output/oneMovie.txt","w+") as fp:
    #lets write the titles first
    fp.write("Title#Views#Stars#Comments#Posted_on\n")


# Lets now write details into the
with open("./output/oneMovie.txt","a+") as fp:
    #lets write the details
    one_movie_details = "#".join((title, views, stars, comments, posted_on))
    fp.write(one_movie_details+"\n")

# Web Pages

The website we are scrapping contain multiple pages - you should be able to notice the change in URL as you scroll. The URLs for different pages appear as follows:
 - Page 1 : https://archive.org/details/opensource_movies or https://archive.org/details/opensource_movies?&sort=-downloads&page=1
 - Page 2  : https://archive.org/details/opensource_movies?&sort=-downloads&page=2
 - Page 3 : https://archive.org/details/opensource_movies?&sort=-downloads&page=3
 - etc
 
I trust you can see the pattern. What changes is the number at the end and therefore we can loop through the pages easily.

## Lets now scrape the first 10 pages (all the movies in every page)

In [43]:
# lets open file to write to
f = open("./output/movies.txt","a+")
# Write the titles
f.write("Title#=View#=Stars#=Comments#=Posted_on\n")
#initialize paging
page_no = 1
while page_no <=10:
    #Lets define the URL - putting in mind the web pagging explained above
    url="https://archive.org/details/opensource_movies?&sort=-downloads&page={}".format(page_no)
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text
    # Parse HTML code for the entire site
    soup = BeautifulSoup(html_content, "lxml")
    #print(soup.prettify()) # print the parsed data of html
    # Movie section
    movies_collection = soup.find("div", attrs={"id": "ikind--downloads"})
    # find all the movies in page one
    aMovie = movies_collection.find_all("div", attrs={"class": "item-ia"})

    for index in range(1,len(aMovie)): #note that movies starts from index 1
        another = aMovie[index]
        #print(another.prettify())
        #getting the title of the movie
        title = (another.find("div",attrs={"class":"ttl"})).text
        title = title.strip()
        # Getting the views count of the movie
        views = another.find_all("h6",attrs={"class":"stat"})[0].nobr.text
        # Stars count
        stars = another.find_all("h6",attrs={"class":"stat"})[1].text
        # example output of above line if "favorite            12 "
        # below line will extract the number from the string above
        stars = re.findall("\d+",stars)[0]
        #comments
        comments =  another.find_all("h6",attrs={"class":"stat"})[2].text
        comments = re.findall("\d+",comments)[0]

        posted_on = ((another.find_all("div",attrs={"class":"hidden-tiles"}))[1]).nobr.text
        one_movie_details = "#=".join((title, views, stars, comments, posted_on))
        f.write(one_movie_details+"\n")
    page_no = page_no + 1
#close the file
f.close()

## Reading .txt file into Pandas DataFrame

In [41]:
df = pd.read_table("./output/movies.txt",sep="#=",engine="python")
df.head()

,Title,View,Stars,Comments,Posted_on
0,Namaz nasıl kılınır?,3M,3,0,"Dec 26, 2019"
1,cCloud TV v0.6 Rise,2.5M,12,1,"Nov 26, 2015"
2,O Shahade Amira Imarata Kavkaz Dokku Abu Usmana,2.3M,11,2,"Jul 20, 2014"
3,israelbicepilp,2.1M,25,9,"May 28, 2011"
4,Medical Videos - 05 Normal Spontsneous vaginal...,1.9M,67,10,"Oct 23, 2010"
